In [1]:
# Import Libraries
import pandas as pd
import json
import os
import sys

In [2]:
# Add the src directory to the Python path
sys.path.append(os.path.abspath('../src'))

from data_processing import load_brent_data, calculate_log_returns, load_event_data
from modeling import load_results # To load previously saved analysis results


In [3]:
print("--- Preparing Data for Dashboard ---")

--- Preparing Data for Dashboard ---


In [5]:
# Define file paths
processed_brent_data_path = '../data/processed/processed_brent_data.csv'
change_point_results_path = '../data/processed/change_point_results.json'
raw_brent_data_path = '../data/raw/BrentOilPrices.csv'
key_events_data_path = '../data/raw/key_events.csv'

In [6]:
# Load historical Brent oil prices (full dataset including original prices)
print(f"Loading raw Brent oil prices from: {raw_brent_data_path}")
brent_raw_df = load_brent_data(raw_brent_data_path)
if brent_raw_df is None:
    print("Failed to load raw Brent oil prices. Exiting.")
    exit()

Loading raw Brent oil prices from: ../data/raw/BrentOilPrices.csv


C:\Users\du\Desktop\Git\brent-oil-price-prediction-using-time-series-data\src\data_processing.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


In [7]:
# Load processed Brent data with log returns
print(f"Loading processed Brent data with log returns from: {processed_brent_data_path}")
brent_processed_df = pd.read_csv(processed_brent_data_path)
brent_processed_df['Date'] = pd.to_datetime(brent_processed_df['Date'])

Loading processed Brent data with log returns from: ../data/processed/processed_brent_data.csv


In [8]:
# Load change point analysis results
print(f"Loading change point analysis results from: {change_point_results_path}")
try:
    change_point_results = load_results(change_point_results_path)
except FileNotFoundError:
    print("Change point results not found. Please run 02_change_point_analysis.ipynb first.")
    exit()

# Load key events data
print(f"Loading key events data from: {key_events_data_path}")
events_df = load_event_data(key_events_data_path)
if events_df is None:
    print("Failed to load key events data. Exiting.")
    exit()

Loading change point analysis results from: ../data/processed/change_point_results.json
Loading key events data from: ../data/raw/key_events.csv


In [9]:
## Prepare data for API endpoints

In [10]:
# 1. Historical Prices (Price and Log_Return)
# Combine raw prices with log returns for a comprehensive time series
full_data_for_dashboard = brent_raw_df.merge(brent_processed_df[['Date', 'Log_Return']], on='Date', how='left')
full_data_for_dashboard['Date'] = full_data_for_dashboard['Date'].dt.strftime('%Y-%m-%d') # Format date for JSON
print("\nSample of historical data for dashboard:")
print(full_data_for_dashboard.head())

# Save to a format easily consumable by Flask (e.g., JSON)
full_data_for_dashboard.to_json('../data/processed/dashboard_historical_data.json', orient='records', date_format='iso')
print(f"Historical data for dashboard saved to ../data/processed/dashboard_historical_data.json")



Sample of historical data for dashboard:
         Date  Price  Log_Return
0  1987-05-20  18.63         NaN
1  1987-05-21  18.45   -0.009709
2  1987-05-22  18.55    0.005405
3  1987-05-25  18.60    0.002692
4  1987-05-26  18.63    0.001612
Historical data for dashboard saved to ../data/processed/dashboard_historical_data.json


In [11]:
# 2. Change Point Details

change_point_date = change_point_results.get('most_probable_change_point_date')
if change_point_date:
    change_point_date_pd = pd.to_datetime(change_point_date)
    # Get the raw price at the change point date for display
    price_at_change_point = brent_raw_df[brent_raw_df['Date'] == change_point_date_pd]['Price'].iloc[0] if not brent_raw_df[brent_raw_df['Date'] == change_point_date_pd].empty else None
    change_point_results['price_at_change_point'] = price_at_change_point

In [12]:
# 3. Key Events
# Format event dates for JSON
events_df['Date'] = events_df['Date'].dt.strftime('%Y-%m-%d')
print("\nSample of event data for dashboard:")
print(events_df.head())
events_df.to_json('../data/processed/dashboard_events_data.json', orient='records', date_format='iso')
print(f"Event data for dashboard saved to ../data/processed/dashboard_events_data.json")

# Combine all results into a single JSON for easy access by frontend, if preferred
dashboard_summary = {
    "change_point_analysis": change_point_results,
    "events": events_df.to_dict(orient='records')
    # Historical data will be loaded separately via an API endpoint due to size
}

with open('../data/processed/dashboard_summary.json', 'w') as f:
    json.dump(dashboard_summary, f, indent=4)
print(f"Dashboard summary data saved to ../data/processed/dashboard_summary.json")

print("\n--- Dashboard Data Preparation Completed ---")
print("These JSON files can now be served by the Flask backend.")


Sample of event data for dashboard:
         Date       EventType  \
0  1990-08-02        Conflict   
1  1997-07-02  Economic Shock   
2  2001-09-11        Conflict   
3  2003-03-20        Conflict   
4  2008-09-15  Economic Shock   

                                    EventDescription  
0              Iraq invades Kuwait (Gulf War begins)  
1                      Asian Financial Crisis begins  
2                             9/11 Terrorist Attacks  
3                                US invasion of Iraq  
4  Lehman Brothers collapse (Global Financial Cri...  
Event data for dashboard saved to ../data/processed/dashboard_events_data.json
Dashboard summary data saved to ../data/processed/dashboard_summary.json

--- Dashboard Data Preparation Completed ---
These JSON files can now be served by the Flask backend.
